In [ ]:
import numpy as np
from IPython.display import display, Math
import matplotlib.pyplot as plt

# LaTeX helper for matrices
def latex_matrix(M):
    if np.shape(M)[0] < 10 and np.shape(M)[1] < 10:
        return (
            r"\begin{bmatrix}"
            + r" \\ ".join(
                [" & ".join(f"{x:.3f}" for x in row) for row in M]
            )
            + r"\end{bmatrix}"
        )
    else:
        return r"\begin{bmatrix} \text{matrix too large to display} \end{bmatrix}"

In [ ]:
# Original matrix
A = np.array([[1, 5],
              [0, 10],
              [2, 3]])

# SVD (thin SVD by default)
U, S, Vt = np.linalg.svd(A)

# Build S matrix
Sigma = np.zeros((U.shape[1], Vt.shape[0]))
Sigma[:len(S), :len(S)] = np.diag(S)

# Reconstruct original matrix
A_new = U @ Sigma @ Vt

# Display everything nicely in LaTeX
display(Math(r"\mathbf{A} = " + latex_matrix(A)))
display(Math(r"\mathbf{U} = " + latex_matrix(U)))
display(Math(r"\boldsymbol{\Sigma} = " + latex_matrix(Sigma)))
display(Math(r"\mathbf{V}^\top = " + latex_matrix(Vt)))
display(Math(r"\mathbf{A}_{\mathrm{new}} = " + latex_matrix(A_new)))

In [ ]:
def gaussian_2d(nx=500, ny=500, sigmax=50, sigmay=100):
    x = np.arange(nx)
    y = np.arange(ny)
    X, Y = np.meshgrid(x, y, indexing="ij")

    x0 = (nx - 1) / 2
    y0 = (ny - 1) / 2

    f = np.exp(-((X - x0)**2 / (2 * sigmax**2) + (Y - y0)**2 / (2 * sigmay**2)))
    return f

# Generate the 500x500 Gaussian matrix
G = gaussian_2d()

# Plot the Gaussian matrix
plt.figure()
plt.imshow(G, origin="lower")
plt.colorbar(label="Amplitude")
plt.xlabel("x")
plt.ylabel("y")
plt.title("2D Gaussian")
plt.show()

In [ ]:
# Original matrix
A = G

# SVD (thin SVD by default)
U, S, Vt = np.linalg.svd(A)

# Build S matrix
Sigma = np.zeros((U.shape[1], Vt.shape[0]))
Sigma[:len(S), :len(S)] = np.diag(S)

# Reconstruct original matrix
A_new = U @ Sigma @ Vt

# Display everything nicely in LaTeX
display(Math(r"\mathbf{A} = " + latex_matrix(A)))
display(Math(r"\mathbf{U} = " + latex_matrix(U)))
display(Math(r"\boldsymbol{\Sigma} = " + latex_matrix(Sigma)))
display(Math(r"\mathbf{V}^\top = " + latex_matrix(Vt)))
display(Math(r"\mathbf{A}_{\mathrm{new}} = " + latex_matrix(A_new)))
